# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [49]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[0, 1, 1, 1],
       [0, 2, 1, 3],
       [0, 3, 1, 3],
       [0, 4, 1, 1],
       [0, 7, 1, 2]])

In [6]:
sparse_train_data[-5:]

array([[5795, 1894,    0,    1],
       [5795, 1986,    0,    1],
       [5795, 2048,    0,    1],
       [5795, 2118,    0,    1],
       [5795, 2129,    0,    1]])

In [7]:
print('No of rows in training file:', sparse_train_data.shape[0])
print('No of rows in test file:', sparse_test_data.shape[0])

No of rows in training file: 259852
No of rows in test file: 116265


In [8]:
print('No of e-mails in training file:', np.unique(sparse_train_data[:, 0]).size)

No of e-mails in training file: 4015


In [9]:
print('No of e-mails in test file:', np.unique(sparse_test_data[:, 0]).size)

No of e-mails in test file: 1722


## How to Create an Empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(VOCAB_SIZE))
len(column_names)

2502

In [11]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [12]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [13]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create a Full Matrix from a Sparse Matrix

In [14]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """Form a full matrix from a sparse matrix. Returns a Pandas DataFrame.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column.
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column.
    cat_idx -- psition of the label (spam is 1, non-spam is 0). Default: 3rd column.
    freq_id -- position of the occurences of the word in the sparse matrix. Default: 4th column.
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(nr_words))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [15]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 47.2 s, sys: 426 ms, total: 47.6 s
Wall time: 54.1 s


In [16]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,1,3,3,1,0,0,2,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,2,3,2,0,1,1,0,2,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,2,0,0,0,2,0,0,8,...,0,0,0,0,0,0,0,0,0,0


In [17]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,2,0,1,1,0,1,0,0,2,...,0,0,0,0,0,0,0,0,0,0
5790,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,0,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [18]:
# Calculate the probability of spam - the percent of spam message in the training dataset
# Store this value in a variable called prob_spam.

# prob_spam = full_train_data.CATEGORY.value_counts()[1]/full_train_data.CATEGORY.size

prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
print('Probability of spam is:', prob_spam)

Probability of spam is: 0.31133250311332505


# Total number of words / Tokens

In [19]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,1,3,3,1,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,3,2,0,1,1,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,3,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,0,0,0,2,0,0,8,2,...,0,0,0,0,0,0,0,0,0,0


In [20]:
email_lenghts = full_train_features.sum(axis=1)
email_lenghts.shape

(4015,)

In [21]:
email_lenghts[:5]

DOC_ID
0    100
1    248
2    126
3      4
4    297
dtype: int64

In [22]:
total_wc = email_lenghts.sum()
total_wc

441044

## Number of Tokens in Spam & Ham Emails

Create a subset of email_lenghts series that only contains the spam messages. Call the subset spam_lenghts. Then count the number of words that occur in spam emails.

Do the same for the non-spam emails. Create a subset called ham_emails. Then count the total number of words that occur in the ham emails.

In [23]:
spam_lenghts = full_train_data[full_train_data.CATEGORY == 1]
spam_lenghts = spam_lenghts.loc[:, spam_lenghts.columns != 'CATEGORY']
total_wc_spam = spam_lenghts.sum(axis=1).sum()
total_wc_spam

185196

In [24]:
spam_lenghts = email_lenghts[full_train_data.CATEGORY == 1]
spam_wc = spam_lenghts.sum()
spam_wc

185196

In [25]:
ham_lenghts = email_lenghts[full_train_data.CATEGORY == 0]
nonspam_wc = ham_lenghts.sum()
nonspam_wc

255848

In [26]:
email_lenghts.shape[0] - spam_lenghts.shape[0] - ham_lenghts.shape[0]

0

In [27]:
spam_wc + nonspam_wc - total_wc

0

In [28]:
# Can you figure out if spam e-mails or non-spam e-mails tend to be longer? Which category contain more tokens?
# Take a guess and then verify your intuition using the dataset.

print('Average nr of words in non-spam emails: {:.0f}'.format(ham_lenghts.mean()))
print('Average nr of words in spam emails: {:.0f}'.format(spam_lenghts.mean()))

Average nr of words in non-spam emails: 93
Average nr of words in spam emails: 148


## Summing the Tokens occuring in a Spam

In [29]:
full_train_features.shape

(4015, 2500)

In [30]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,1,3,3,1,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,3,2,0,1,1,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,3,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,0,0,0,2,0,0,8,2,...,0,0,0,0,0,0,0,0,0,0


In [31]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,4,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1887,3,0,2,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,4,0,0,2,2,2,3,7,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,0,0,3,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1895,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
train_spam_tokens.shape

(1250, 2500)

In [33]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [34]:
summed_spam_tokens.tail()

2495     9
2496     3
2497     2
2498     1
2499    17
dtype: int64

## Summing the Tokens occuring in Ham

In [35]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1 # Laplace smoothing tecnique

In [36]:
summed_ham_tokens.shape

(2500,)

In [37]:
summed_ham_tokens.tail()

2495    24
2496    36
2497    37
2498    30
2499    13
dtype: int64

In [38]:
train_ham_tokens[2499].sum() + 1

13

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [39]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012249
1    0.005163
2    0.006654
3    0.010517
4    0.006985
dtype: float64

In [40]:
prob_tokens_spam.sum()

1.0000000000000002

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [41]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0

## P(Token) - Probability that Token Occurs

In [42]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all.sum()

1.0000000000000002

# Save the Trained Model

In [44]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

Create a full matrix from the sparse_test_data. Time the function call. How long does it take? Separate the features and the target values. Save this as separate .txt file: a TEST_TARGET_FILE and a TEST_FEATURE_MATRIX file.

In [46]:
sparse_test_data.shape

(116265, 4)

In [47]:
%%time
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

CPU times: user 24.5 s, sys: 281 ms, total: 24.8 s
Wall time: 26.2 s


In [48]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [50]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)